In [179]:
import uproot
import awkward as ak
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import hist
import vector
import os
import subprocess
import gc
print("uproot version", uproot.__version__)
print("awkward version", ak.__version__)
print("numpy version", np.__version__)
print("matplotlib version", matplotlib.__version__)
print("hist version", hist.__version__)
print("vector version", vector.__version__)
#print("os version", os.__version__)
#print("subprocess version", subprocess.__version__)
#print("gc version", gc.__version__)

uproot version 5.0.12
awkward version 2.4.3
numpy version 1.25.2
matplotlib version 3.8.0
hist version 2.7.2
vector version 1.1.1


In [163]:
vector.register_awkward() 

In [146]:
DATATYPE = "mc"
assert((DATATYPE == "mc") or (DATATYPE == "data"))
BASEDIR = "/pbs/throng/training/nantes-m2-rps-exp/data" # basedir where to look for runXXX.DATATYPE.root files
IS_MC = True if DATATYPE == "mc" else False

In [147]:
def data_file_path(run,is_mc=IS_MC,dest=BASEDIR):
    datatype="mc" if is_mc else "data"
    print({dest},"/run",{run},".",{datatype},".root")
    return f"{dest}/run{run}.{datatype}.root"

In [180]:
RUNS = [290323, 290327, 290848, 291361, 291360, 291362, 290853, 290860, 291373, 290374, 290375, 291399,
               291400, 290894, 290895, 290404, 291943, 291944, 291948, 291953, 290932, 290423, 291447, 290935, 
               290425, 290427, 291451, 291453, 291976, 291982, 290456, 290458, 290459, 291482, 291485, 290975, 
               290980, 290469, 292012, 291002, 291003, 291004, 291005, 290501, 292040, 292060, 292061, 292062, 
               291041, 290539, 290540, 292075, 292077, 292080, 290549, 290553, 291590, 292106, 292108, 292109, 
               292115, 290590, 291618, 291622, 291624, 292140, 290612, 292160, 292162, 292163, 292164, 292166, 
               290632, 291657, 292168, 292192, 290658, 290660, 291690, 291692, 291694, 291698, 291706, 290687, 
               290692, 290696, 290699, 292242, 292265, 291755, 292269, 292270, 291760, 292273, 292274, 290742, 
               291769, 291263, 290764, 290766, 291283, 291284, 291285, 291795, 291796, 290776, 291803, 290787]

run2=RUNS[0:2]
SAMPLE_RUNS = sorted(run2)
#SAMPLE_RUNS = sorted(RUNS)
print(len(SAMPLE_RUNS))

2


In [181]:
file = uproot.open(data_file_path(SAMPLE_RUNS[0],IS_MC))
events = file["eventsTree"]
events.show()
events.num_entries

gen = file["genTree"]
gen.show()


{'/pbs/throng/training/nantes-m2-rps-exp/data'} /run {290323} . {'mc'} .root
name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
runNumber            | int32_t                  | AsDtype('>i4')
xVtx                 | double                   | AsDtype('>f8')
yVtx                 | double                   | AsDtype('>f8')
zVtx                 | double                   | AsDtype('>f8')
isCINT               | bool                     | AsDtype('bool')
isCMSL               | bool                     | AsDtype('bool')
isCMSH               | bool                     | AsDtype('bool')
isCMLL               | bool                     | AsDtype('bool')
isCMUL               | bool                     | AsDtype('bool')
nMuons               | int32_t                  | AsDtype('>i4')
Muon_E               | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
Muon_Px              | st

In [188]:
def mag(Px, Py, Pz):
    return np.sqrt(Px*Px + Py*Py + Pz*Pz)

def rest_mass(E, Px, Py, Pz):
    P = mag(Px, Py, Pz)
    return np.sqrt(np.abs(E*E - P*P))


def getTracks(events):
    return ak.zip({"px":events["Muon_Px"],
                       "py":events["Muon_Py"],
                       "pz":events["Muon_Pz"],
                       "E":events["Muon_E"],
                       "charge":events["Muon_Charge"],
                       "thetaAbs":events["Muon_thetaAbs"],
                       "matched":events["Muon_matchedTrgThreshold"],
                       "PDGCode" : events["Muon_MCPDGCode"]},
                    with_name='Momentum4D')

def getGen(gen):
    return ak.zip({"Muon_Gen_Label":gen["Muon_GenLabel"],
                   "Mother_PDG_Code":gen["Muon_GenMotherPDGCode"]},
                   with_name='Momentum4D')

def scan(dataDescription, 
              hMag:hist.Hist, hPhi:hist.Hist, hEta:hist.Hist, hPt:hist.Hist, hMass:hist.Hist, hMass_OS:hist.Hist, hMass_LS:hist.Hist,
              eventSelector = lambda x:[True]*len(x),
              trackSelector = lambda x:[True]*len(x), 
              genSelector = lambda x:[True]*len(x),
              verbose:bool = False):
    Entries = goodEntries = 0
    for batch in uproot.iterate(dataDescription,
                            ["isCINT", "isCMUL", "isCMSL", "nMuons", "Muon_Px", "Muon_Py", "Muon_Pz", "Muon_E", "Muon_Charge", "Muon_thetaAbs", "Muon_matchedTrgThreshold", "Muon_MCLabel ", "Muon_MCPDGCode"],                                
                                report = True):
        events = batch[0] # batch[1] is the report info
        if len(events) < 1000:
            print("something is wrong",batch[1]) # this is a protection for some corrupted input data files 
            break
            
        goodEvents = events[eventSelector(events)] 
        tracks = getTracks(events)
        goodTracks = tracks[trackSelector(tracks)]

        
        hMag.fill(ak.flatten(goodTracks.p))
        hPhi.fill(ak.flatten(goodTracks.phi))
        hEta.fill(ak.flatten(goodTracks.eta))
        hPt.fill(ak.flatten(goodTracks.pt))
        hMass.fill(ak.flatten(goodTracks.mass))
        
        
        pairs = ak.combinations(goodTracks, 2)
        Px = pairs["0"].px + pairs["1"].px
        Py = pairs["0"].py + pairs["1"].py
        Pz = pairs["0"].pz + pairs["1"].pz
        E = pairs["0"].E + pairs["1"].E
        charge = pairs["0"].charge + pairs["1"].charge
        
        
        Pair2 = ak.zip({"px":Px,
                       "py":Py,
                       "pz":Pz,
                       "E":E,
                       "charge":charge},
                    with_name='Momentum4D')
        
        condition = ((pairs["0"].PDGCode==13) & (pairs["1"].PDGCode==-13)) | ((pairs["0"].PDGCode==-13) & (pairs["1"].PDGCode==13))
        A=pairs[condition]
        Px = A["0"].px + A["1"].px
        Py = A["0"].py + A["1"].py
        Pz = A["0"].pz + A["1"].pz
        E = A["0"].E + A["1"].E
        charge = A["0"].charge + A["1"].charge
            
        pair_muons = ak.zip({"px":Px,
                             "py":Py,
                             "pz":Pz,
                             "E":E,
                             "charge":charge},
                        with_name='Momentum4D')
        
       # labels = [pairs["0"].muon_label , pairs["1"].muon_label]
        
        #hMass_OS.fill(ak.flatten(Pair2[Pair2.charge==0].mass))
        #hMass_LS.fill(ak.flatten(Pair2[Pair2.charge!=0].mass))
        hMass_OS.fill(ak.flatten(pair_muons[charge==0].mass))
        hMass_LS.fill(ak.flatten(pair_muons[charge!=0].mass))
        
        
        
        Entries += len(events)
        goodEntries += len(ak.flatten(goodTracks))/2

        
        if verbose:
            print("-----------------------------------------------------------")
            print(batch[1], "\nEntries: ", Entries, "\nJ/Psi: ", int(goodEntries), "\nPDG: ", PDG)
            gc.collect()
    return Entries, goodEntries, PDG

In [151]:
%%time

hMag = hist.Hist(hist.axis.Regular(bins = 100, start = 0, stop = 100, name = '$|p|$'))
hPhi = hist.Hist(hist.axis.Regular(bins = 200, start = -22/7, stop = 22/7, name = '$\phi$'))

hEta = hist.Hist(hist.axis.Regular(bins = 100, start = -6, stop = 0, name = '$\eta$'))
hPt = hist.Hist(hist.axis.Regular(bins = 100, start = 0, stop = 100, name = '$p_T$'))
hMass = hist.Hist(hist.axis.Regular(bins = 100, start = 0, stop = 0.2, name = '$m_{\mu}$'))

hMass_OS = hist.Hist(hist.axis.Regular(bins = 100, start = 1.5, stop = 5, name = '$m_{\mu \mu}$'))
hMass_LS = hist.Hist(hist.axis.Regular(bins = 100, start = 1.5, stop = 5, name = '$m_{\mu \mu}$'))

hMass_OS_cut = hist.Hist(hist.axis.Regular(bins = 100, start = 1.5, stop = 5, name = '$m_{\mu \mu}$'))
hMass_LS_cut = hist.Hist(hist.axis.Regular(bins = 100, start = 1.5, stop = 5, name = '$m_{\mu \mu}$'))

hMass_OS_test = hist.Hist(hist.axis.Regular(bins = 100, start = 1.5, stop = 5, name = '$m_{\mu \mu}$'))
hMass_LS_test = hist.Hist(hist.axis.Regular(bins = 100, start = 1.5, stop = 5, name = '$m_{\mu \mu}$'))


CPU times: user 1.17 ms, sys: 0 ns, total: 1.17 ms
Wall time: 1.19 ms


In [189]:
PDG = []
NTotalJPsi =0
NJPsi=[]
Entries=0
goodEntries =0
for run in SAMPLE_RUNS:
    a, b, c = scan(dataDescription = f"/pbs/throng/training/nantes-m2-rps-exp/data/run{run}.mc.root:eventsTree",
          hMag = hMag, hPhi = hPhi, hEta = hEta, hPt = hPt, hMass = hMass, hMass_OS = hMass_OS_test, hMass_LS = hMass_LS_test,
          eventSelector = lambda x: x["isCMUL"] == True, 
          trackSelector = lambda x: (((x.thetaAbs < 3) & (x.eta > -4)) & (x.eta < -2.5)), verbose = True)
    Entries += a;
    goodEntries += b;
    NJPsi.append(int(b))
    NTotalJPsi += goodEntries
    
    #print(a,b)
print("-----------------------------------------------------------", "\nEntries: ", Entries, "\nJ/Psi: ", int(goodEntries), "\nPDG: ", PDG)
print(NTotalJPsi)
print(NJPsi)
print(len(NJPsi))


-----------------------------------------------------------
<Report start=0 stop=70000 source='/pbs/throng/training/nantes-m2-rps-exp/data/run290323.mc.root:/eventsTree;1'> 
Entries:  70000 
J/Psi:  7349 
PDG:  []
-----------------------------------------------------------
<Report start=0 stop=6000 source='/pbs/throng/training/nantes-m2-rps-exp/data/run290327.mc.root:/eventsTree;1'> 
Entries:  6000 
J/Psi:  637 
PDG:  []
----------------------------------------------------------- 
Entries:  76000 
J/Psi:  7986 
PDG:  []
15336.0
[7349, 637]
2


In [ ]:
"""[7349, 637, 4147, 2115, 8523, 3198, 8423, 3131, 2102, 2088, 
15879, 9487, 11735, 5374, 4273, 7338, 5301, 8440, 3151, 2105, 
726, 8461, 8532, 11723, 5300, 4182, 7378, 3145, 4222, 4183, 
4207, 9496, 5230, 15709, 3185, 1041, 5318, 3176, 7387, 2131, 
10509, 7343, 10656, 3142, 12699, 17155, 4626, 8059, 2324, 10275, 
2300, 8033, 2318, 4589, 4552, 5696, 4493, 14568, 9202, 2261, 
7357, 4285, 3254, 6296, 5300, 2109, 4342, 4317, 4297, 7464, 
6312, 219, 7375, 1048, 2116, 3198, 7420, 5348, 9583, 13760, 
7471, 5279, 6387, 8435, 6379, 4223, 12685, 7446, 3168, 5306, 
4321, 4243, 3070, 10572, 729, 13890, 810, 5310, 824, 4146, 
6388, 824, 3190, 3173, 973, 4204, 10743, 4188]"""

In [ ]:
hMag.plot(color = "blue", label = "no cut")
plt.yscale('log')
plt.grid(True)
plt.legend()

In [ ]:
hMass_OS.plot(color = "blue", label = "no cut")
#hMass_cut.plot(color = "red", label = "cut")
plt.yscale('log')
plt.ylabel("# of tracks")
plt.grid(True)
plt.legend()